In [1]:
'''
# 라이브러리 정의
 - 시스템 라이브러리
 - 오디오 관련 라이브러리
 - 연산 및 Plot 관련 라이브러리 
 - 샤용자 라이브러리
'''

# Import Systems 
import struct
import io
import os
import sys
import math
import platform
import time

# Import Audio
import pyaudio
import sounddevice
import librosa
import soundfile

import numpy as np
import scipy
import scipy.signal as sig
import matplotlib.pyplot as plt

# User Libraries
import pyOssWavfile
import pyRoomAcoustic as room
import pyOssDebug as dbg
import pyOssFilter
import pyOssPyAudio

%matplotlib tk


In [2]:
set_device_type = 'asio'

if 'darwin' in platform.system():
    IDX = pyaudio.paCoreAudio           # = 5
elif 'linux' in platform.system():
    IDX = pyaudio.paALSA
else:
    if set_device_type == 'asio':
        IDX = pyaudio.paASIO            # = 3
    elif set_device_type == 'wasapi':
        IDX = pyaudio.paWASAPI          # = 13
    elif set_device_type == 'wdm':
        IDX = pyaudio.paWDMKS           # = 11
    else:
        IDX = pyaudio.paMME             # = 2

print('Index of Device Type:', IDX)

# List of Device Names IDX 
dev_name = pyOssPyAudio.ju_get_device_name(IDX)
print(dev_name)


Index of Device Type: 3
['ASIO4ALL v2', 'Yamaha Steinberg USB ASIO']


In [ ]:
# 장치선택: Windows MME

tgt_dev_in = dev_name[0]        # Microsoft 사운드 매퍼 - Input
tgt_dev_out = dev_name[3]       # Microsoft 사운드 매퍼 - Output

dev_in = pyOssPyAudio.ju_get_device_info(IDX, tgt_dev_in)
st_dev_in = pyOssPyAudio.CAudioDeviceInfo()

st_dev_in.index = dev_in['index']
st_dev_in.name = dev_in['name']
st_dev_in.maxInCh = dev_in['maxInputChannels']
st_dev_in.maxOutCh = dev_in['maxOutputChannels']
st_dev_in.fs = int(dev_in['defaultSampleRate'])

# dev_in_index = dev_in['index']
# dev_in_name = dev_in['name']
# dev_in_ch = dev_in['maxInputChannels']
# dev_in_fs = int(dev_in['defaultSampleRate'])

dev_out = pyOssPyAudio.ju_get_device_info(IDX, tgt_dev_out)

dev_out_index = dev_out['index']
dev_out_name = dev_out['name']
dev_out_ch = dev_out['maxInputChannels']
dev_out_fs = int(dev_out['defaultSampleRate'])


In [3]:
# 장치 선택: ASIO
# tgt_asio_dev_name = 'ASIO4ALL v2'
tgt_asio_dev_name = 'Yamaha Steinberg USB ASIO'
# tgt_asio_dev_name = 'Steinberg USB ASIO'

dev_asio = pyOssPyAudio.ju_get_device_info(IDX, tgt_asio_dev_name)

dev_asio_index = dev_asio['index']
dev_asio_name = dev_asio['name']
dev_asio_ch_in = dev_asio['maxInputChannels']
dev_asio_fs = dev_asio['defaultSampleRate']

print('dev_asio: ', dev_asio)
print(dev_asio_index, dev_asio_name, dev_asio_ch_in, dev_asio_fs)


dev_asio:  {'index': 17, 'structVersion': 2, 'name': 'Yamaha Steinberg USB ASIO', 'hostApi': 2, 'maxInputChannels': 6, 'maxOutputChannels': 6, 'defaultLowInputLatency': 0.011609977324263039, 'defaultLowOutputLatency': 0.011609977324263039, 'defaultHighInputLatency': 0.011609977324263039, 'defaultHighOutputLatency': 0.011609977324263039, 'defaultSampleRate': 44100.0}
17 Yamaha Steinberg USB ASIO 6 44100.0


In [4]:
# Record & save audio data from Mic using ASIO Device

pa = pyaudio.PyAudio()

# create matplotlib figure and axes
fig, ax = plt.subplots(1, figsize=(15, 7))

FORMAT = pa.get_format_from_width(2)
CH = dev_asio_ch_in
CHUNK = 1024 * 2
RATE = np.int(dev_asio_fs)            # device 정보에서 읽어오면 됨
BUFFER = CHUNK * 2 * CH
RECORD_SECONDS = 10

print(pa.is_format_supported(RATE, 
            input_device=dev_asio_index,
            input_channels=CH,
            input_format=FORMAT,
     ))

stream1 = pa.open(
    format = FORMAT,
    channels = CH,
    rate = RATE,
    input = True,
    output = False,
    frames_per_buffer = BUFFER,
    input_device_index = dev_asio_index
)

# stream2 = pa.open(
#     format = FORMAT,
#     channels = 2,
#     rate = RATE,
#     input = True,
#     output = False,
#     frames_per_buffer = BUFFER,
#     input_device_index = dev_asio_index
# )


True


In [5]:
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)

# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()
cnt_ch = 0

while True:
    # binary data
    data = stream1.read(CHUNK, exception_on_overflow=False)
    if CH > 1:
        data_temp = data[0:BUFFER]
    #     data_ch1 = data[0:(2*CHUNK-1)]
    #     data_ch2 = data[(2*CHUNK):(2*(2*CHUNK)-1)]
    else:
        data_temp = data

    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data_temp)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break



stream started


NameError: name 'TclError' is not defined

# Record input audio data
print("* recording")

frames1 = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data1 = stream1.read(CHUNK)
    frames1.append(data1)

print("* done recording")

In [10]:
# Stream Stop Process 

stream1.stop_stream()
stream1.close()
pa.terminate()          #


In [ ]:
# Not really sure what b'' means in BYTE STRING but numpy needs it 
# just like wave did...
framesAll = b''.join(frames1)

# Use numpy to format data and reshape.  
# PyAudio output from stream.read() is interlaced.
result = np.fromstring(framesAll, dtype=np.int16)
chunk_length = np.int(len(result) / CH)
result = np.reshape(result, (chunk_length, CH))


# Write multi-channel .wav file with SciPy
pyOssWavfile.write('pyaudio_rec.wav', RATE, result)
print("* done save wav file")